# Get MF-DFA on starling amplitude envelope
#### Date: 8/31/2022, JX
> Use MF-DFA on /mnt/cube/j8xing/starling_rhythm_mfdfa/data/starling_zenodo_2019_amp_envs.pickle

In [1]:
from starling_rhythm.utils.paths import PROCESSED_DIR

## Import dataframe

In [2]:
import pandas as pd

In [3]:
SAVE_PATH = PROCESSED_DIR / 'starling_zenodo_2019_amp_envs200.pickle'
ae_df = pd.read_pickle(SAVE_PATH)

In [4]:
ae_df.head()

,bID,wav_file,amp_env
0,b1053,2017-11-13_16-29-28-410000.wav,"[0.009873399510979652, 0.015623422339558601, 0..."
1,b1053,2017-11-16_15-17-52-520000.wav,"[0.0003380597336217761, 0.00034135428722947836..."
2,b1053,2017-11-20_14-15-32-360000.wav,"[2.641892910003662, 2.6070051193237305, 2.5947..."
3,b1053,2017-11-20_12-40-56-130000.wav,"[3.971318801632151e-05, 3.945435310015455e-05,..."
4,b1053,2017-11-15_15-30-01-000000.wav,"[0.005851297173649073, 0.49159008264541626, 1...."


## Pull out MF-DFA

> 1. Filter time-series to have compact support
2. Determine time windows

In [5]:
import numpy as np
from starling_rhythm.mfdfa import hurst_expo
from joblib import Parallel, delayed
from tqdm.autonotebook import tqdm

/home/AD/j8xing/anaconda3/envs/starling_rhythm/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
n_jobs = 36 ## set up joblib parameters
verbose = 0

In [7]:
with Parallel(n_jobs = n_jobs, verbose = verbose) as parallel:
    h_expos = parallel(
        delayed(hurst_expo)(
            amp_env
        ) 
        for amp_env in tqdm(ae_df.amp_env.values, desc = "Retrieving Hurst Exponents for all Amplitude Envelopes.")
    )

Retrieving Hurst Exponents for all Amplitude Envelopes.: 100%|██████████| 10499/10499 [03:02<00:00, 57.42it/s]


In [8]:
## Get MF_range
MF_range = []
for h_expo in h_expos:
    MF_range.append(max(h_expo) - min(h_expo))

In [9]:
ae_df['h_expo'] = h_expos
ae_df['MF_range'] = MF_range

In [10]:
ae_df

,bID,wav_file,amp_env,h_expo,MF_range
0,b1053,2017-11-13_16-29-28-410000.wav,"[0.009873399510979652, 0.015623422339558601, 0...","[1.528071866963577, 1.0370111976118286, 0.8437...",1.011097
1,b1053,2017-11-16_15-17-52-520000.wav,"[0.0003380597336217761, 0.00034135428722947836...","[1.1938116698829084, 1.0984076594608625, 1.032...",0.351719
2,b1053,2017-11-20_14-15-32-360000.wav,"[2.641892910003662, 2.6070051193237305, 2.5947...","[1.1806910145575205, 1.0637405119967855, 0.977...",0.426110
3,b1053,2017-11-20_12-40-56-130000.wav,"[3.971318801632151e-05, 3.945435310015455e-05,...","[1.1870676604975956, 1.0644852178050632, 0.983...",0.416832
4,b1053,2017-11-15_15-30-01-000000.wav,"[0.005851297173649073, 0.49159008264541626, 1....","[1.1330045795095076, 1.0547100658878008, 0.984...",0.376517
...,...,...,...,...,...
10495,b1145,2017-11-09_13-26-01-740000.wav,"[0.0003850629145745188, 0.0003816374228335917,...","[1.180004894319496, 1.0197033102020214, 0.9132...",0.545500
10496,b1145,2017-11-07_17-30-06-880000.wav,"[0.00022515977616421878, 0.0001997244544327259...","[1.2047430508822017, 1.0585859494621113, 0.943...",0.611907
10497,b1145,2017-11-09_09-34-49-120000.wav,"[0.004488207399845123, 1.2879040241241455, 3.4...","[1.28912933912076, 1.1004097813293756, 0.96728...",0.661470
10498,b1145,2017-11-09_09-26-04-120000.wav,"[0.00029243348399177194, 0.0002883640117943287...","[1.260354642359515, 1.0466296263441792, 0.9064...",0.669632


## Pickle dataframe

In [11]:
from starling_rhythm.utils.paths import PROCESSED_DIR, ensure_dir

In [12]:
SAVE_PATH = PROCESSED_DIR / 'starling_zenodo_2019_amp_envs_MF200.pickle'
ensure_dir(SAVE_PATH)

In [13]:
ae_df.to_pickle(SAVE_PATH)